In [66]:
import tensorflow as tf
import tensorflow_datasets as tfds

tf.debugging.set_log_device_placement(True)

In [67]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12830079897850581952
xla_global_id: -1
]


데이터 셋 로드 (TFDS에서 가져온 EMNIST)

In [68]:
(etfds_train, etfds_test), etfds_info = tfds.load("emnist", split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)
print(etfds_info)
print(etfds_train)

def normalize_img_ascii_lable(image, label):
    ascii_Lb = label
    if ascii_Lb >= 10:
        ascii_Lb += 8
    elif ascii_Lb >= 43:
        ascii_Lb += 6
    ascii_Lb += 15
    return tf.cast(image, tf.float32) / 255., ascii_Lb

etfds_train = etfds_train.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_train = etfds_train.cache()
etfds_train = etfds_train.shuffle(etfds_info.splits['train'].num_examples)
etfds_train = etfds_train.batch(128)
etfds_train = etfds_train.prefetch(tf.data.experimental.AUTOTUNE)
print(etfds_train)

etfds_test = etfds_test.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_test = etfds_test.batch(128)
etfds_test = etfds_test.cache()
etfds_test = etfds_test.prefetch(tf.data.experimental.AUTOTUNE)

tfds.core.DatasetInfo(
    name='emnist',
    full_name='emnist/byclass/3.0.0',
    description="""
    The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19 and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset.
    
    Note: Like the original EMNIST data, images provided here are inverted horizontally and rotated 90 anti-clockwise. You can use `tf.transpose` within `ds.map` to convert the images to a human-friendlier format.
    """,
    config_description="""
    EMNIST ByClass
    """,
    homepage='https://www.nist.gov/itl/products-and-services/emnist-dataset',
    data_path='C:\\Users\\Administrator\\tensorflow_datasets\\emnist\\byclass\\3.0.0',
    file_format=tfrecord,
    download_size=535.73 MiB,
    dataset_size=349.16 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=62),
  

직접 만든 데이터 셋 로드

In [69]:
import pathlib

data_dir = pathlib.Path('..\dataset')

trainFG = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(28,28),
    batch_size=16,
    label_mode='int',
    color_mode="grayscale"
)



testFG = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(28,28),
    batch_size=16,
    label_mode='int',
    color_mode="grayscale"
)

def normalize_myimg_ascii_lable(image, label):
    return tf.cast(image, tf.float32) / 255., tf.cast(int(label), dtype=tf.int64)

trainFG = trainFG.map(normalize_myimg_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
trainFG = trainFG.cache()
trainFG = trainFG.prefetch(tf.data.experimental.AUTOTUNE)
print(trainFG)

testFG = testFG.map(normalize_myimg_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
testFG = testFG.cache()
testFG = testFG.prefetch(tf.data.experimental.AUTOTUNE)


Found 1520 files belonging to 95 classes.
Using 1216 files for training.
Found 1520 files belonging to 95 classes.
Using 304 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


모델 세팅

In [70]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(95, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 784)               0         
                                                                 
 dense_18 (Dense)            (None, 512)               401920    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 95)                48735     
                                                                 
Total params: 450655 (1.72 MB)
Trainable params: 450655 (1.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


훈련시키기

In [71]:
model.fit(etfds_train, epochs=3)
model.fit(trainFG, epochs=25)

Epoch 1/3
5453/5453 [==============================] - 300s 52ms/step - loss: 0.6863 - accuracy: 0.7854
Epoch 2/3
5453/5453 [==============================] - 270s 49ms/step - loss: 0.5132 - accuracy: 0.8262
Epoch 3/3
5453/5453 [==============================] - 269s 49ms/step - loss: 0.4847 - accuracy: 0.8329
Epoch 1/25
76/76 [==============================] - 2s 25ms/step - loss: 5.4204 - accuracy: 0.0641
Epoch 2/25
76/76 [==============================] - 2s 23ms/step - loss: 2.4067 - accuracy: 0.4770
Epoch 3/25
76/76 [==============================] - 2s 23ms/step - loss: 1.2463 - accuracy: 0.7426
Epoch 4/25
76/76 [==============================] - 2s 23ms/step - loss: 0.6875 - accuracy: 0.8709
Epoch 5/25
76/76 [==============================] - 2s 23ms/step - loss: 0.3969 - accuracy: 0.9375
Epoch 6/25
76/76 [==============================] - 2s 23ms/step - loss: 0.2667 - accuracy: 0.9729
Epoch 7/25
76/76 [==============================] - 2s 23ms/step - loss: 0.1804 - accuracy: 0.

모델 저장하기

In [72]:
model.save('spemnist.h5')

확인하기

In [73]:
#model.evaluate(etfds_test, verbose=2)
model.evaluate(testFG, verbose=2)

19/19 - 0s - loss: 2.3641 - accuracy: 0.4868 - 368ms/epoch - 19ms/step


[2.3641421794891357, 0.4868420958518982]